In [1]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'coco-outdoor-wavelet'
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "coco-cropped-Outdoor"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'sandbox', 'raw-data','coco')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [3]:
file_list = [os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}"))
file_names[:5]

['000000335177.jpg',
 '000000278705.jpg',
 '000000568981.jpg',
 '000000092416.jpg',
 '000000173830.jpg']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [6]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


toy-coco-cropped-Outdoor
._toy-coco-cropped-Outdoor



In [7]:
import numpy as np

def npz_opener_pickle(path):
    image = Image.open(path).convert('RGB')
    arr = np.array(image).astype(np.float32)
    jitter = np.random.uniform(-0.5, 0.5, arr.shape)
    arr += jitter
    arr = np.clip(arr, 0, 255)
    return arr

# Wavelet

In [8]:
FINAL_DATA_NAME = 'coco-outdoor-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "sandbox", "raw-data", "coco", f"toy-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "sandbox", "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [ ]:
TRANSFORM = "wavelet"
channel = "red"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-19259.096, -19165.588, -18495.984, ...,  19158.129,  20503.637,
        20818.31 ], shape=(2446,), dtype=float32), np.int64(3): array([-11188.516, -10392.083, -10239.648, ...,  10450.108,  10530.21 ,
        11950.772], shape=(9784,), dtype=float32), np.int64(4): array([-6124.298 , -5833.559 , -5626.8105, ...,  6262.9136,  6299.09  ,
        6600.382 ], shape=(39136,), dtype=float32), np.int64(5): array([-3272.7366, -3217.279 , -3204.5447, ...,  2951.105 ,  2989.04  ,
        3182.3494], shape=(100000,), dtype=float32), np.int64(6): array([-1872.535 , -1683.3845, -1629.1167, ...,  1566.377 ,  1647.2451,
        1848.1818], shape=(100000,), dtype=float32), np.int64(7): array([-950.33307, -857.24194, -833.4833 , ...,  869.6318 ,  987.3166 ,
       1005.174  ], shape=(100000,), dtype=float32), np.int64(8): array([-478.55368, -421.2182 , -408.9013 , ...,  405.9977 ,  418.0344 ,
        480.25632], shape=(100000,), dtype=float32), np.int64(9): array([-254.86

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-15188.622, -14982.646, -14880.948, ...,  15315.091,  16848.74 ,
        20400.133], shape=(2446,), dtype=float32), np.int64(3): array([-10839.301, -10380.639, -10374.067, ...,   9680.847,   9706.972,
        11956.701], shape=(9784,), dtype=float32), np.int64(4): array([-6885.585 , -5911.5225, -5693.3926, ...,  5691.756 ,  5693.115 ,
        6515.33  ], shape=(39136,), dtype=float32), np.int64(5): array([-3458.7192, -3206.842 , -3182.3455, ...,  3245.37  ,  3413.8237,
        3518.4849], shape=(100000,), dtype=float32), np.int64(6): array([-1743.8726, -1671.4539, -1619.5424, ...,  1594.2013,  1644.232 ,
        1697.441 ], shape=(100000,), dtype=float32), np.int64(7): array([-928.20575, -852.58405, -822.7509 , ...,  826.851  ,  853.3226 ,
        975.524  ], shape=(100000,), dtype=float32), np.int64(8): array([-499.7656 , -419.54602, -406.72797, ...,  403.475  ,  417.8548 ,
        480.72662], shape=(100000,), dtype=float32), np.int64(9): array([-254.82

In [ ]:
TRANSFORM = "wavelet"
channel = "green"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-20245.676, -19686.979, -19627.152, ...,  20321.568,  20503.916,
        21014.857], shape=(2446,), dtype=float32), np.int64(3): array([-11345.502, -10547.392,  -9891.067, ...,  10354.834,  10430.316,
        10767.508], shape=(9784,), dtype=float32), np.int64(4): array([-6208.51  , -5481.024 , -5310.303 , ...,  6167.0435,  6262.6553,
        6600.7656], shape=(39136,), dtype=float32), np.int64(5): array([-3302.074 , -3172.6914, -3170.0044, ...,  3039.9048,  3081.7722,
        3197.9546], shape=(100000,), dtype=float32), np.int64(6): array([-1882.6305, -1705.424 , -1632.1721, ...,  1559.0328,  1590.8412,
        1845.7057], shape=(100000,), dtype=float32), np.int64(7): array([-946.31604, -854.3783 , -827.62164, ...,  864.6023 ,  994.39294,
       1001.1417 ], shape=(100000,), dtype=float32), np.int64(8): array([-473.1598 , -422.60107, -409.84268, ...,  408.83832,  420.07513,
        480.38675], shape=(100000,), dtype=float32), np.int64(9): array([-254.72

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-15017.322, -14822.194, -13834.171, ...,  15381.168,  16505.91 ,
        22862.508], shape=(2446,), dtype=float32), np.int64(3): array([-10380.948 , -10008.695 ,  -9973.551 , ...,  10228.3955,
        11162.02  ,  12200.279 ], shape=(9784,), dtype=float32), np.int64(4): array([-5561.006 , -5444.5947, -5443.6177, ...,  5783.563 ,  5880.4688,
        5925.144 ], shape=(39136,), dtype=float32), np.int64(5): array([-3346.7402, -3270.9043, -3189.063 , ...,  3297.836 ,  3424.2952,
        3543.9485], shape=(100000,), dtype=float32), np.int64(6): array([-1797.0463, -1715.9141, -1623.4104, ...,  1623.5222,  1674.8535,
        1796.1827], shape=(100000,), dtype=float32), np.int64(7): array([-927.5306 , -840.86414, -817.57776, ...,  833.0665 ,  865.07874,
        941.0327 ], shape=(100000,), dtype=float32), np.int64(8): array([-496.66235, -427.11072, -410.058  , ...,  405.06876,  417.81473,
        473.08957], shape=(100000,), dtype=float32), np.int64(9): array([-

In [ ]:
TRANSFORM = "wavelet"
channel = "blue"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-20960.207, -20495.621, -20480.123, ...,  20643.627,  20793.748,
        22358.604], shape=(2446,), dtype=float32), np.int64(3): array([-11730.287, -10498.101, -10266.637, ...,  10977.15 ,  11197.2  ,
        12348.804], shape=(9784,), dtype=float32), np.int64(4): array([-6297.9595, -6006.769 , -5952.9707, ...,  6440.356 ,  6600.2925,
        6699.411 ], shape=(39136,), dtype=float32), np.int64(5): array([-3371.2244, -3096.3096, -3094.7905, ...,  3111.8323,  3115.4915,
        3209.3257], shape=(100000,), dtype=float32), np.int64(6): array([-1771.9393, -1627.7697, -1568.1664, ...,  1566.5753,  1654.5873,
        1855.4291], shape=(100000,), dtype=float32), np.int64(7): array([-924.93823, -841.90137, -817.2251 , ...,  848.3376 ,  989.9903 ,
       1008.96674], shape=(100000,), dtype=float32), np.int64(8): array([-475.0304 , -418.4434 , -405.07098, ...,  404.3105 ,  416.19315,
        483.16223], shape=(100000,), dtype=float32), np.int64(9): array([-253.51

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-18012.498, -15959.873, -15821.478, ...,  15986.565,  16314.797,
        24955.012], shape=(2446,), dtype=float32), np.int64(3): array([-10382.622, -10380.72 ,  -9259.255, ...,  10019.688,  11603.592,
        12318.254], shape=(9784,), dtype=float32), np.int64(4): array([-5711.103 , -5598.871 , -5486.5693, ...,  6093.2646,  6131.593 ,
        6167.28  ], shape=(39136,), dtype=float32), np.int64(5): array([-3314.54  , -2983.0366, -2964.9946, ...,  3192.2756,  3322.932 ,
        3413.1592], shape=(100000,), dtype=float32), np.int64(6): array([-1884.147 , -1660.9617, -1587.2683, ...,  1575.6835,  1595.1849,
        1782.2495], shape=(100000,), dtype=float32), np.int64(7): array([-927.56213, -827.19904, -790.5832 , ...,  809.268  ,  844.90375,
        931.78766], shape=(100000,), dtype=float32), np.int64(8): array([-499.12378, -424.6758 , -406.85135, ...,  398.94794,  412.64224,
        464.90692], shape=(100000,), dtype=float32), np.int64(9): array([-254.87

In [9]:
TRANSFORM = "wavelet"
channel = "gray"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-19802.484, -19648.938, -19584.682, ...,  19897.523,  20501.799,
        20737.477], shape=(2446,), dtype=float32), np.int64(3): array([-11341.274, -10494.2  ,  -9820.746, ...,  10167.977,  10413.218,
        10598.259], shape=(9784,), dtype=float32), np.int64(4): array([-5998.9683, -5499.8877, -5151.819 , ...,  6072.8247,  6262.427 ,
        6599.9136], shape=(39136,), dtype=float32), np.int64(5): array([-3280.3245, -3169.5886, -3147.337 , ...,  2933.7307,  2948.397 ,
        3141.189 ], shape=(100000,), dtype=float32), np.int64(6): array([-1866.7701, -1637.1066, -1571.6494, ...,  1546.3054,  1585.7024,
        1847.6471], shape=(100000,), dtype=float32), np.int64(7): array([-928.52844, -845.6403 , -819.7389 , ...,  853.00653,  993.2215 ,
        996.14886], shape=(100000,), dtype=float32), np.int64(8): array([-474.64325, -418.39862, -405.06866, ...,  406.31616,  417.09332,
        480.55566], shape=(100000,), dtype=float32), np.int64(9): array([-254.44

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-15064.251, -13838.759, -13499.741, ...,  15116.192,  16274.763,
        22363.076], shape=(2446,), dtype=float32), np.int64(3): array([-10379.79 , -10127.814,  -9949.482, ...,   9389.099,   9558.925,
        12139.925], shape=(9784,), dtype=float32), np.int64(4): array([-5463.1035, -5413.224 , -5344.7715, ...,  5770.4663,  5795.575 ,
        5932.093 ], shape=(39136,), dtype=float32), np.int64(5): array([-3325.2043, -3015.0757, -2938.896 , ...,  3165.458 ,  3419.411 ,
        3513.8735], shape=(100000,), dtype=float32), np.int64(6): array([-1790.275 , -1630.92  , -1594.9766, ...,  1583.8788,  1643.5669,
        1728.9775], shape=(100000,), dtype=float32), np.int64(7): array([-927.5774 , -834.90295, -797.7751 , ...,  818.54553,  849.7706 ,
        939.40155], shape=(100000,), dtype=float32), np.int64(8): array([-497.4475 , -422.3033 , -406.52994, ...,  400.7926 ,  414.4386 ,
        471.1863 ], shape=(100000,), dtype=float32), np.int64(9): array([-254.79